In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from functions import *

%load_ext autoreload
%autoreload 2

In [2]:
## Underlying data
underlying_ticker = 'BOVA11'
underlying = yf.download((underlying_ticker + '.SA'), period='12mo')[['Adj Close']]
last_price = underlying.iloc[-1].iloc[0]
last_price

[*********************100%%**********************]  1 of 1 completed


117.3499984741211

## Options Data Collection

In [3]:
## Collect front month data to calculate expected move in the operation period  
exp_date_front = '2024-07-19'
            
options_chain_front = get_options_chain(underlying_ticker, exp_date_front)
options_chain_front['abs_Distance'] = options_chain_front['Distance'].abs() 
options_chain_front[options_chain_front['Moneyness'] == 'ATM'].sort_values('abs_Distance').head(2)

# options_chain_front[options_chain_front['Moneyness'] == 'OTM'].sort_values('abs_Distance').head(14).sort_values('Strike')

,Option,Type,E/A,Moneyness,Strike,Distance,Premium,volume,abs_Distance
19,BOVAG117,CALL,E,ATM,117.0,-0.11,3.28,592585.29,0.11
99,BOVAS117,PUT,E,ATM,117.0,-0.11,1.74,524795.81,0.11


In [4]:
## Collect back month data to get the strikes and premium of the operation
exp_date_back = '2024-08-16'
            
options_chain_back = get_options_chain(underlying_ticker, exp_date_back)
options_chain_back['abs_Distance'] = options_chain_back['Distance'].abs() 
options_chain_back[options_chain_back['Moneyness'] == 'ATM'].sort_values('abs_Distance').head(2)

# options_chain_back[options_chain_back['Moneyness'] == 'OTM'].sort_values('abs_Distance').head(14).sort_values('Strike')

,Option,Type,E/A,Moneyness,Strike,Distance,Premium,volume,abs_Distance
17,BOVAH117,CALL,E,ATM,117.0,-0.11,4.75,2342.05,0.11
93,BOVAT117,PUT,E,ATM,117.0,-0.11,2.10,9787.10,0.11


## Calculating ATM Straddle
This is made to define the expected move within the lifetime of the Options

In [5]:
## calculate expected move within operation period
atm_straddle, lower, upper = atm_short_straddle(options_chain_front, last_price)

ATM Straddle: 5.02
Range Expectation using ATM Straddle 112.33 e 122.37


## Collecting Underlying Data 

In [6]:
## Dates to calculate opration period using 
# operation time in days 
days = 29

In [7]:
rets = underlying.pct_change().dropna(axis=0)
vol = annualize_vol(rets, days).iloc[0]  
last_price = round(underlying.tail(1).reset_index(drop=True).iloc[0].iloc[0],2)

print(f'Volatility: {vol:.2%} in last {days} days')

Volatility: 4.91% in last 29 days


In [8]:
vol_based_short(last_price, vol, 1 )

Range Expectation using ATM Straddle 111.58 e 123.12


(111.58420876382209, 123.1157912361779)

In [9]:
(atm_straddle / last_price * 100) * 1.25

5.347251810822326

In [10]:
## ATM straddle as Volatility approximation
# .153 == implied vol
straddle_aproxx = 0.8 * last_price * .153 * np.sqrt(days / 252)
atm_straddle, straddle_aproxx, (straddle_aproxx/atm_straddle)

(5.02, 4.872627818222348, 0.9706429916777587)

## Selecting Options to construct Iron Condor

In [11]:
# Define the strikes using front month ATM straddle as a proxy for expected move
long_put_strike, short_put_strike, short_call_strike, long_call_strike = select_iron_condor_strikes(options_chain_front, last_price, wing_width=3)
# long_put, short_put, short_call, long_call

ATM Straddle: 5.02
Range Expectation using ATM Straddle 112.33 e 122.37


In [12]:
## print strikes
print(f'long_put: {long_put_strike:.2f}')
print(f'short_put: {short_put_strike:.2f}')
print(f'short_call: {short_call_strike:.2f}')
print(f'long_call: {long_call_strike:.2f}')

long_put: 109.00
short_put: 112.00
short_call: 122.00
long_call: 125.00


In [13]:
## Collecting the premiuns from the options_chain_back
long_put = options_chain_back[(options_chain_back['Strike'] == long_put_strike) & (options_chain_back['Type'] == 'PUT')]
short_put = options_chain_back[(options_chain_back['Strike'] == short_put_strike) & (options_chain_back['Type'] == 'PUT')]
short_call = options_chain_back[(options_chain_back['Strike'] == short_call_strike) & (options_chain_back['Type'] == 'CALL')]
long_call = options_chain_back[(options_chain_back['Strike'] == long_call_strike) & (options_chain_back['Type'] == 'CALL')] 

long_put_premium = long_put['Premium'].iloc[0] 
short_put_premium = short_put['Premium'].iloc[0] 
short_call_premium = short_call['Premium'].iloc[0] 
long_call_premium = long_call['Premium'].iloc[0]

## print premium 
print(f'long_put: {long_put_strike:.2f}, {long_put_premium:.2f}')
print(f'short_put: {short_put_strike:.2f}, {short_put_premium:.2f}')
print(f'short_call: {short_call_strike:.2f}, {short_call_premium:.2f}')
print(f'long_call: {long_call_strike:.2f}, {long_call_premium:.2f}')

long_put: 109.00, 0.66
short_put: 112.00, 1.05
short_call: 122.00, 2.07
long_call: 125.00, 1.10


In [14]:
## Calculate how much each leg (puts vs calls) contribute to total credit 
put_spread =  short_put_premium - long_put_premium 
call_spread =  short_call_premium - long_call_premium
print(f'Call Spread: {call_spread:.2f}\nPut Spread: {put_spread:.2f}')
call_spread_pct = call_spread / (call_spread+put_spread)
print(f'Call side pct: {call_spread_pct:.2%}')

## if call_spread_pct > .70 of total credit, unless there is some bullish bias, consider only doing the call bear spread
if call_spread_pct > .70:
    print(f'Consider only doing the call bear spread')
else:    
    print(f'both sides have good premium')

Call Spread: 0.97
Put Spread: 0.39
Call side pct: 71.32%
Consider only doing the call bear spread


In [15]:
## Calculates total money risked, legs lenght, total credit, managed Tp and managed ROIC
max_loss, gain_range, credit_received, profit, roc_cost, leg_width = iron_condor(
    options_chain_back,
    long_put_strike,
    short_put_strike,
    short_call_strike,
    long_call_strike
        )

Position Risk: 16.40
Gain Range: 10.00
Credit Received/Max Profit: $13.60
Managed Take Profit: $7.07
Managed ROIC (net): 43.12%
